In [1]:
import warnings
warnings.filterwarnings("ignore")

## Research various skew related topics
* **This requires plotly 4.0 or greater** *

### Analyse options volatility in futures contract, creating graphs that show vertical skew historically over time, and also show skew vs futures prices over time


In [2]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr

# importlib.reload(build_db)

### important global variables

In [3]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
pga = db_info.get_db_info()


  sec_db


In [4]:
df_iv_final = pd.read_csv('./temp_folder/df_iv_final_ES.csv')
df_iv_skew = pd.read_csv('./temp_folder/df_iv_skew_ES.csv')

## Define methods to graph skew, and create subplots with separate legends.  

#### This only works with plotly 4..0 and above 

In [5]:
# use this method to generate x/y scatter or bar graphs
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None):
    '''
    Return a plotly Figure that you can use with iplot, to produce
    multi-axis x/y scatter and bar graphs
    
    Example:
xvals = list(range(1,101))
y1vals = np.arange(101,201,1)
y2vals = np.arange(11,1,-.1)
y3vals = y2vals * .5
             
df = pd.DataFrame({'my_x_vals':xvals, 
                    'yvals_1':y1vals,
                    'yvals_2':y2vals,
                    'yvals_3':y3vals
                  })
fig = plotly_plot(df_in=df,x_column='my_x_vals',
                    plot_title = "example graph",
                    y_left_label='main y vals',
                    y_right_label='alt y vals',
                    yaxis2_cols = ['yvals_2','yvals_3'],
                    number_of_ticks_display=25)
iplot(fig)

    
    :param df_in: data frame with a single x, and multiple y values to graph
    :param x_column: the column in df_in that holds the x-axis values
    :param plot_title: title of plot 
    :param y_left_label:
    :param y_right_label:
    :param bar_plot: If True, plot bars, other wise, plot x/y scatter line
    :param figsize: 
    :param number_of_ticks_display: Number of x axis major ticks to display
    :param yaxis2_cols: the dataframe columns that hold the y values for the second y axis, if any
    :param x_value_labels: if you want alternate labels for the x axis, specify them here
    '''
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    return fig

def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

In [6]:
xvals = list(range(1,101))
y1vals = np.arange(101,201,1)
y2vals = np.arange(11,1,-.1)
y3vals = y2vals * .5
             
df = pd.DataFrame({'my_x_vals':xvals, 
                    'yvals_1':y1vals,
                    'yvals_2':y2vals,
                    'yvals_3':y3vals
                  })
fig = plotly_plot(df_in=df,x_column='my_x_vals',
                    plot_title = "example graph",
                    y_left_label='main y vals',
                    y_right_label='alt y vals',
                    yaxis2_cols = ['yvals_2','yvals_3'],
                    number_of_ticks_display=25)
iplot(fig)



In [7]:
def create_skew_per_date_df(df):
    '''
    Find the first settle_date whose count of rows is equal to max count of rows.
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]
    # get just that symbol's data
    df12 = df[df.symbol==contract]
    df_counts = df12[['settle_date','moneyness']].groupby('settle_date',as_index=False).count()
    max_count = df_counts.moneyness.max()
    first_max_count_settle_date = df_counts[df_counts.moneyness==max_count].iloc[0].settle_date
    
    df_ret = df12[df12.settle_date==first_max_count_settle_date][['moneyness']]
    all_settle_dates = sorted(df_counts.settle_date.unique())
    for settle_date in all_settle_dates:
        df_temp = df12[df12.settle_date==settle_date][['moneyness','vol_skew']]
        df_ret = df_ret.merge(df_temp,on='moneyness',how='outer')
        df_ret = df_ret.rename(columns={'vol_skew':str(settle_date)})
    df_ret = df_ret.sort_values('moneyness')
    df_ret.moneyness = df_ret.moneyness.round(4)
    return df_ret


In [8]:
def graph_skew(df_iv_final,do_plot=False):
    '''
    Graph skew for ONLY ONE symbol.
    If df_iv_final contains more than one symbol, we will only graph the first symbol in the DataFrames    
    '''
    # get the first symbol (which should be the only symbol)
    contract = df_iv_final.symbol.unique()[0]
    # get just that symbol's data and only days that have sufficient skew data
    dft = df_iv_final[df_iv_final.symbol==contract]
    dft_count = dft[['settle_date','symbol']].groupby('settle_date',as_index=False).count()
    valid_settle_dates = dft_count[dft_count.symbol>2].settle_date.unique()
    dft = dft[dft.settle_date.isin(valid_settle_dates)]
    dfp = create_skew_per_date_df(dft)
    
    settle_dates = sorted([c for c in dfp.columns.values if c != 'moneyness'])
    splits = list(np.arange(5,len(settle_dates),5))
    settle_date_groups = np.split(np.array(settle_dates),splits)
    ret_figs = []
    for sdg in settle_date_groups:
        sdg_sorted = [str(c) for c in sorted(sdg)]
        cols = ['moneyness']+list(sdg_sorted)
        dfp_sub = dfp[cols]
        t = f'{contract} {sdg[0]} - {sdg[-1]}' 
        f = plotly_plot(dfp_sub,x_column='moneyness',plot_title=t,y_left_label='vol skew')
        ret_figs.append(f)
        if do_plot:
            iplot(f)
    return ret_figs

#### Do the same plots as above, but using a grid

In [9]:

def graph_skew_subplot_quad(fig_group,rows=1,cols=2):
    '''
    Use subplots to output the results of the method graph_skew above
    '''
    f1 = make_subplots(rows=rows, cols=cols,  
        shared_yaxes=False,
        shared_xaxes=False,                       
        subplot_titles=[fig_group[i]['layout'].title.text for i in range(len(fig_group))],
        horizontal_spacing=0.05,
        vertical_spacing=0.11,                       
        print_grid=False)

    pl_width=450*cols 
    pl_height=400*rows
    title = 'Skew plots<br>'

    f1.update_layout(title=title,                                 
        font= Font(family="Open Sans, sans-serif"),
        showlegend=True,     
        hovermode='x',  
        autosize=True,       
        width=pl_width,       
        height=pl_height,
        plot_bgcolor='#EFECEA', 
        bargap=0.05,
        margin=Margin(
                      l=5,
                      r=5,
                      b=55,
                      t=50
        )
    )    

    for i in range(len(fig_group)):
        x = int(i/2) + 1
        y = i % 2 + 1
        f = fig_group[i]
        l = f.layout
        if y > 1:
            l.yaxis.title=''
        try:
            yaxis = f'yaxis{i+1}'
            xaxis = f'xaxis{i+1}'
            if i < 10:
                yaxis = yaxis.replace('1','') 
                xaxis = xaxis.replace('1','') 
            gname = f'{x,y}'#rfs[i]['layout'].title
            for d in f.data:
                data_y = f'y{i+1}'.replace('1','') 
                d['yaxis']=data_y
                d['legendgroup'] =  gname
                d['name'] = f"{d.name}"
                f1.add_trace(d,x,y)
                f1.update_xaxes(patch=l.xaxis,row=x,col=y)
                f1.update_yaxes(patch=l.yaxis,row=x,col=y)                
        except Exception as e:
            print(f'graph_skew_subplots ERRORS: {str(e)}')
    return f1


def graph_skew_subplots(df,rows=2,cols=2):
    fig_list = graph_skew(df)
    n = rows*cols   
    # using list comprehension 
    fig_groups = [fig_list[i*n:(i + 1)*n] for i in range((len(fig_list) + n - 1) // n )]  
    for fig_group in fig_groups:
        iplot(graph_skew_subplot_quad(fig_group,rows=rows,cols=cols))


### Graph skew changes historically, per day

In [10]:
SYMBOL_TO_RESEARCH = 'ES'
# all_contracts = pga.get_sql(f"select symbol from {opttab} where substring(symbol,1,2)='{SYMBOL_TO_RESEARCH}'").symbol.unique()
all_contracts = df_iv_final[df_iv_final.symbol.str.slice(0,2)=='ES'].symbol.unique()
all_contracts 

array(['ESM11', 'ESU11', 'ESZ11', 'ESH12', 'ESM12', 'ESU12', 'ESZ12',
       'ESH13', 'ESM13', 'ESU13', 'ESZ13', 'ESH14', 'ESM14', 'ESU14',
       'ESZ14', 'ESH15', 'ESM15', 'ESU15', 'ESZ15', 'ESH16', 'ESM16',
       'ESU16', 'ESZ16', 'ESH17', 'ESM17', 'ESU17', 'ESZ17', 'ESH18',
       'ESM18', 'ESU18', 'ESZ18', 'ESH19', 'ESM19', 'ESU19', 'ESZ19',
       'ESH20', 'ESM20'], dtype=object)

In [11]:
grid_plot=True
clist = [c for c in all_contracts if (c[:2]==f'{SYMBOL_TO_RESEARCH}') & (c[-2:]=='19')]
volcols = ['symbol','settle_date','moneyness','vol_skew']
dfivf = df_iv_final[volcols].groupby(['symbol','settle_date','moneyness'],as_index=False).mean()

for c in clist:
    dft = dfivf[dfivf.symbol==c]
    if len(dft)<=0:
        print(f'no data for symbol {c}')
        continue
    if grid_plot:
        graph_skew_subplots(dfivf[dfivf.symbol==c],rows=1,cols=2)
    else:
        rls = graph_skew(dfivf[dfivf.symbol==c],do_plot=True)
        

### Use the implied vol skew dataframes to graph changes in vol skew vs price

In [12]:
y = 10
syms = [s for s in all_contracts if  (int(s[-2:]) >= y)]

for sym in syms:
    df_fut = pga.get_sql(f"select symbol,settle_date,close from {futtab} where symbol = '{sym}'")
    df_year = df_iv_skew[df_iv_skew.symbol==sym]
    df_year['2dn_2up'] = df_year['-0.2'] - df_year['0.2']
    df_year = df_year[['settle_date','2dn_2up']]
    df_year = df_year.merge(df_fut[['settle_date','close']],on='settle_date',how='inner')
    iplot(plotly_plot(df_year,x_column='settle_date',
            yaxis2_cols=['close'],y_left_label='skew',y_right_label=sym,
                     plot_title=f'{sym} close vs skew -0.2, 0.2',
                     number_of_ticks_display=15))
    

2020-03-10 20:34:40,506 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [13]:
y = 10
syms = [s for s in all_contracts if  (int(s[-2:]) >= y)]

for sym in syms:
    df_fut = pga.get_sql(f"select symbol,settle_date,close from {futtab} where symbol = '{sym}'")
    df_year = df_iv_final[['settle_date','atm_iv']][df_iv_final.symbol==sym]
    df_year = df_year.groupby(['settle_date'],as_index=False).mean()
    df_year.atm_iv = df_year.atm_iv.round(4)
    df_year = df_year.merge(df_fut[['settle_date','close']],on='settle_date',how='inner')
    iplot(plotly_plot(df_year,x_column='settle_date',
            yaxis2_cols=['close'],y_left_label='atm_iv',y_right_label=sym,
                     plot_title=f'{sym} close vs atm implied vol',
                     number_of_ticks_display=15))
    

In [14]:
def get_postgres_data(contract,pga):
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    usql = f"select * from {futtab} where symbol='{contract}';"
    dfu = pga.get_sql(usql)
    # Merge options and futures data
    df = dfo.merge(dfu,how='inner',on=['symbol','settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    return df,df_expiry_dates

In [15]:
USE_PYVOL = True
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates,contract):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df2 = df2[(((df2.pc=='C' )& (df2.strike>=df2.close_y)) | ((df2.pc=='P' ) & (df2.strike<df2.close_y)))  & (df2.symbol.str.contains(contract))]
    df2 = df2[df2.contract_num==2]
    phigh = df2.close_y.max()
    plow = df2.close_y.min()
    high_strike = round(phigh * 1.3)
    low_strike = round(plow * .7)
    df2 = df2[(df2.strike>=low_strike) & (df2.strike<=high_strike)]

    df9 = df2[df2.symbol==contract]
    df9 = df9.merge(df_expiry_dates.rename(columns={'settle_date':'expiry'}),on='symbol',how='inner')
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



In [16]:
def get_even_moneyness_strikes(df10):
    # define amounts around the money which will help create strikes to add
    moneyness = np.arange(.7,1.4,.05).round(6)
    # define columns on which to execute groupby
#     gb_cols = ['symbol','settle_date','pc','contract_num','dte','close_y']
    gb_cols = ['symbol','settle_date','contract_num','dte','close_y']
    # define function used in groupby.apply to create strikes and iv's at those strikes
    #   where the strikes are an even amount from the money 
    #   (like .7, .8, ... 1, 1.1, 1.2, etc)
    def _add_even_moneyness_strikes(df):
        # get underlying from first row (the groupby makes them all the same)
        r = df.iloc[0]
        underlying = r.close_y
        # create new rows to append to df, using only the gb_cols
        df_ret1 = df.iloc[:len(moneyness)][gb_cols].copy()
        # add nan iv's !!!! MUST BE np.nan - NOT None
        df_ret1['iv'] = np.nan
        # add new strikes
        df_ret1['strike'] = moneyness * underlying
        # append the new strikes
        dfa = df.append(df_ret1,ignore_index=True,sort=True).copy()
        df_ret2 = dfa.sort_values(['symbol','settle_date','pc','strike'])
        df_ret2 = df_ret2.drop_duplicates(subset='strike')
        # set the index to the strike so that interpolate works
        df_ret2.index = df_ret2.strike
        # create interpolated iv's
        df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
        # reset the index
        df_ret2.index = list(range(len(df_ret2)))
        return df_ret2

    # start here
    df11 = df10.groupby(gb_cols).apply(_add_even_moneyness_strikes).copy()
    df11.index = list(range(len(df11)))
    df11['moneyness'] = df11.strike / df11.close_y
    df11.moneyness = df11.moneyness.round(4)

    df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())].copy()
    df12.moneyness  = df12.moneyness - 1
    df12.index = list(range(len(df12)))
    df12_atm = df12[df12.moneyness==0][['symbol','settle_date','pc','iv']]
    df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
    
    df12_atm = df12_atm.drop_duplicates()
    df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')
    df12.moneyness = df12.moneyness.round(4)
    df12['vol_skew'] = (df12.iv - df12.atm_iv).round(4)
    return df12



## END